# TP1 – Taller de Programación (UBA 2025)

**Grupo 4:** Ángel Zapata, David Robalino y Federico Kisza  
**Profesora:** María Noelia Romero  
**Materia:** Taller de Programación  
**Carrera:** Maestría en Economía Aplicada (UBA)  
**Fecha:** 4 de octubre de 2025

---

## 🎯 Objetivo

El presente trabajo tiene como objetivo aplicar los métodos de análisis de datos y programación en Python vistos en clase, utilizando librerías como `pandas`, `numpy`, `matplotlib` y `seaborn`, para realizar un análisis exploratorio sobre la base asignada.


In [1]:
# ========================================================
# SECCIÓN 1 - Importar librerías y configurar entorno
# ========================================================

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Estilos y opciones generales
plt.style.use('seaborn-v0_8-darkgrid')
pd.set_option('display.max_columns', None)

# Definir rutas base del trabajo
BASE = os.path.abspath(".")  # Carpeta actual
DATOS = os.path.join(BASE, "datos")
FIGS  = os.path.join(BASE, "figuras")

# Crear carpetas si no existen
os.makedirs(DATOS, exist_ok=True)
os.makedirs(FIGS, exist_ok=True)

# Mostrar rutas configuradas
print("Rutas configuradas correctamente:")
print("📂 BASE  ->", BASE)
print("📂 DATOS ->", DATOS)
print("📂 FIGS  ->", FIGS)

Rutas configuradas correctamente:
📂 BASE  -> C:\Users\econz\OneDrive\Documentos\Lecturas libros\Cursos UBA\Programación\Grupo4_UBA_2025\TP1
📂 DATOS -> C:\Users\econz\OneDrive\Documentos\Lecturas libros\Cursos UBA\Programación\Grupo4_UBA_2025\TP1\datos
📂 FIGS  -> C:\Users\econz\OneDrive\Documentos\Lecturas libros\Cursos UBA\Programación\Grupo4_UBA_2025\TP1\figuras


In [2]:
# ========================================================
# SECCIÓN 2 - Carga de datos EPH (2005 y 2025)
# ========================================================

# En esta sección vamos a:
# 1️⃣ Verificar si los archivos de la EPH están en la carpeta /datos
# 2️⃣ Cargar los archivos en formato .dta (Stata) y .xls/.xlsx (Excel)
# 3️⃣ Mostrar la cantidad de observaciones y columnas de cada base

import os
import glob
import pandas as pd

# Definir ruta base
DATOS = r"C:\Users\econz\OneDrive\Documentos\Lecturas libros\Cursos UBA\Programación\Grupo4_UBA_2025\TP1\datos"

# Definimos las rutas de los archivos
archivo_2005 = os.path.join(DATOS, "Individual_t105.dta")

# Buscar automáticamente el archivo de 2025 (sea .xls o .xlsx)
archivos_2025 = glob.glob(os.path.join(DATOS, "usu_individual_T125.*"))
if archivos_2025:
    archivo_2025 = archivos_2025[0]
    print("✅ Archivo 2025 encontrado:", archivo_2025)
else:
    print("⚠️ No se encuentra el archivo 2025.")
    archivo_2025 = None

# Verificar si los archivos existen
if not os.path.exists(archivo_2005):
    print("⚠️ No se encuentra el archivo 2005 en:", archivo_2005)
else:
    print("✅ Archivo 2005 encontrado.")

# Cargar los datos (si existen)
try:
    eph2005 = pd.read_stata(archivo_2005)
    if archivo_2025:
        eph2025 = pd.read_excel(archivo_2025)
    print("\n✅ Bases cargadas correctamente.")
except Exception as e:
    print("\n❌ Error al cargar las bases:", e)

# Mostrar información básica de cada base
try:
    print("\n--- Información de la base 2005 ---")
    print(eph2005.shape)
    print(eph2005.head(3))

    print("\n--- Información de la base 2025 ---")
    print(eph2025.shape)
    print(eph2025.head(3))
except:
    print("⚠️ No se puede mostrar información, verifica las rutas o nombres de archivo.")

✅ Archivo 2025 encontrado: C:\Users\econz\OneDrive\Documentos\Lecturas libros\Cursos UBA\Programación\Grupo4_UBA_2025\TP1\datos\usu_individual_T125.xlsx
✅ Archivo 2005 encontrado.

✅ Bases cargadas correctamente.

--- Información de la base 2005 ---
(47030, 176)
     CODUSU  nro_hogar componente h15    ano4       trimestre  \
0  125098          1.0        1.0  Sí  2005.0  1er. Trimestre   
1  125188          1.0        1.0  Sí  2005.0  1er. Trimestre   
2  125188          1.0        2.0  Sí  2005.0  1er. Trimestre   

              region mas_500              aglomerado  pondera            ch03  \
0  Gran Buenos Aires       S  Ciudad de Buenos Aires   1017.0            Jefe   
1  Gran Buenos Aires       S  Ciudad de Buenos Aires   1062.0            Jefe   
2  Gran Buenos Aires       S  Ciudad de Buenos Aires   1062.0  Cónyuge/Pareja   

    ch04  ch06    ch07                        ch08 ch09  \
0  Varón  80.0   Viudo  Obra social (incluye PAMI)   Sí   
1  Varón  26.0  Casado    No paga

In [3]:
# ========================================================
# SECCIÓN 3 - Filtrado de Región Patagónica y unión de bases (versión integrada con aporte de Fede)
# ========================================================

# En esta versión integramos el enfoque propuesto por Fede:
# 1️⃣ Filtramos ambas bases por Región Patagónica
# 2️⃣ Seleccionamos variables clave comunes
# 3️⃣ Estandarizamos nombres y concatenamos

import pandas as pd
from pathlib import Path

# Definimos carpeta de trabajo (donde están las bases)
DATOS = Path.cwd() / "datos"

# --- Filtrado de Región Patagónica ---
# Confirmamos los valores únicos en cada base
print("Regiones disponibles:")
print("2005 →", eph2005["region"].unique())
print("2025 →", eph2025["REGION"].unique())

# Filtramos la región Patagónica
eph2005_pat = eph2005[eph2005["region"] == "Patagónica"].copy()
eph2025_pat = eph2025[eph2025["REGION"] == 43].copy()

# Agregamos columna año
eph2005_pat["anio"] = 2005
eph2025_pat["anio"] = 2025

# --- Selección de variables comunes (según propuesta de Fede) ---
variables = [
    "CODUSU", "NRO_HOGAR", "COMPONENTE", "CH04", "CH06", "CH07", "CH08",
    "NIVEL_ED", "ESTADO", "CAT_INAC", "IPCF", "ITF", "anio"
]

# Estandarizamos nombres a minúsculas
eph2005_pat.columns = eph2005_pat.columns.str.lower()
eph2025_pat.columns = eph2025_pat.columns.str.lower()

# Seleccionamos columnas comunes
cols_comunes = [v.lower() for v in variables if v.lower() in eph2005_pat.columns and v.lower() in eph2025_pat.columns]

# Unimos bases
eph_pat_total = pd.concat(
    [eph2005_pat[cols_comunes], eph2025_pat[cols_comunes]],
    ignore_index=True
)

print(f"✅ Bases combinadas: {eph_pat_total.shape[0]} observaciones, {eph_pat_total.shape[1]} variables")
eph_pat_total.head(3)


Regiones disponibles:
2005 → ['Gran Buenos Aires', 'NOA', 'NEA', 'Cuyo', 'Pampeana', 'Patagónica']
Categories (6, object): ['Gran Buenos Aires' < 'NOA' < 'NEA' < 'Cuyo' < 'Pampeana' < 'Patagónica']
2025 → [43 41 42 40 44  1]
✅ Bases combinadas: 17032 observaciones, 13 variables


,codusu,nro_hogar,componente,ch04,ch06,ch07,ch08,nivel_ed,estado,cat_inac,ipcf,itf,anio
0,125814,1.0,1.0,Varón,46.0,Casado,Obra social (incluye PAMI),Secundaria Incompleta,Ocupado,0.0,480.0,2400.0,2005
1,125814,1.0,2.0,Mujer,32.0,Casado,Obra social (incluye PAMI),Secundaria Incompleta,Inactivo,Ama de casa,480.0,2400.0,2005
2,125814,1.0,3.0,Varón,14.0,Soltero,Obra social (incluye PAMI),Primaria Completa,Inactivo,Estudiante,480.0,2400.0,2005
